In [2]:
import numpy as np
import pandas as pd
import spacy

from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from spacy.lang.es import Spanish

In [ ]:
parser = Spanish()

In [ ]:
!python -m spacy download es_core_news_md

In [3]:
nlp = spacy.load("es_core_news_sm")

In [4]:
def tokenize(text):
    lda_tokens = []
    tokens = nlp(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        #elif token.like_url:
        #    lda_tokens.append('URL')
        #elif token.orth_.startswith('@'):
        #    lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token)
    return lda_tokens


In [5]:
tokens = tokenize("la casa es verde y grande")

In [11]:
tokens[5].pos_

'ADJ'

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

In [13]:
# stemmer = SnowballStemmer('spanish')
stop_words_es = set(stopwords.words("spanish"))
stop_words_es.update(['proyecto', 'investigacion', 'nuevo', 'nueva', 'estudio', 'desarrollo', 'digital', 'objeto', 'información', 'informacion', 'parte', 'documento', 'dato'])
tokenizer = RegexpTokenizer(r'\w+')


In [21]:
def prepare_text_for_lda(text):
    #tokens = tokenize(text)
    doc = nlp(text)
    tokens =  []
    #tokens = [token for token in tokens if len(token.text) > 4]
    tokens = [token for token in doc if token.lemma_.lower() not in stop_words_es]
    tokens = [token for token in tokens if token.pos_ in  ['NOUN', 'VERB', 'ADJ']]
    return tokens

In [15]:
def prepare_text_for_lda(text):
    tokens = nlp(text)
    for token in tokens:
        if token.lemma_.lower() not in stop_words_es:
             if token.pos_ == 'PROPN' and token.lower_=='datos':
                    print(token.sent)
                    print(token.i)
        #if token.pos_ in  ['NOUN', 'VERB', 'ADJ']:
        #    print("caca")
          
    return tokens

In [16]:
def read_data(path):
    """
    :param path: of desir the xlsx file
    :return: String that contains all project descriptions
    """
    df = pd.read_excel(path, 0)
    filtered_df = df.replace(np.nan, '-', regex=True)
    data = (filtered_df['Resumen'].str.lower().tolist())
    return data

In [23]:
descriptions = read_data("data/HUMANIDADES DIGITALES_APROBADO_REVISADO_MT.xlsx")


In [24]:
text_data = []
for line in descriptions:
    tokens = prepare_text_for_lda(line)
    text_data.append(tokens)

In [25]:
text_data

[[actual,
  dispersion,
  aplicacion,
  conceptos,
  herramientas,
  humanidades,
  plantea,
  oordinacion,
  ambito,
  letras,
  hispanas,
  campo,
  fin,
  establecer,
  procedimientos,
  comunes,
  niveles,
  incluidos,
  conceptuales,
  metodologicos,
  visibilidad,
  tecnologicos,
  objetivos,
  buscados,
  inciden,
  planos,
  reconocimiento,
  esfuerzos,
  resultados,
  desarrollo,
  instrumentos,
  trabajo,
  apertura,
  lineas,
  existentes,
  registros],
 [consolidacion,
  medios,
  entorno,
  cultural,
  bytes,
  soportes,
  conocidos,
  cine,
  libros,
  formas,
  artisticas,
  necesario,
  marco,
  estudio,
  dirigir,
  ambito,
  humanidades,
  marco,
  comenzando,
  lleva,
  años,
  reivindicandose,
  comunidad,
  academica,
  internacional,
  anglosajona,
  interseccion,
  inevitable,
  necesaria,
  informatica,
  medios,
  humanidades,
  vivimos,
  ademas,
  tiempo,
  convergencia,
  mediatica,
  cultural,
  -utilizando,
  marcado,
  diferentes,
  medios,
  conocidos,
 

In [26]:
text_data2 = []
for line in text_data:
    word_list = []
    for word in line:
        word_list.append(word.lemma_)
        list(set(word_list))
    text_data2.append(word_list)

text_data2

[['actual',
  'dispersion',
  'aplicacion',
  'concepto',
  'herramienta',
  'humanidad',
  'plantear',
  'oordinacion',
  'ambito',
  'letra',
  'hispano',
  'campar',
  'fin',
  'establecer',
  'procedimiento',
  'común',
  'nivelar',
  'incluir',
  'conceptual',
  'metodologicos',
  'visibilidad',
  'tecnologicos',
  'objetivo',
  'buscar',
  'incidir',
  'plano',
  'reconocimiento',
  'esfuerzo',
  'resultar',
  'desarrollar',
  'instrumento',
  'trabajar',
  'apertura',
  'linear',
  'existente',
  'registro'],
 ['consolidacion',
  'medio',
  'entornar',
  'cultural',
  'byte',
  'soporte',
  'conocido',
  'cine',
  'libro',
  'formar',
  'artisticas',
  'necesario',
  'marcar',
  'estudiar',
  'dirigir',
  'ambito',
  'humanidad',
  'marcar',
  'comenzar',
  'llevar',
  'año',
  'reivindicandose',
  'comunidad',
  'academica',
  'internacional',
  'anglosajón',
  'interseccion',
  'inevitable',
  'necesario',
  'informatica',
  'medio',
  'humanidad',
  'vivir',
  'ademar',
  'ti

### LDA with Gensim

In [ ]:
from gensim import corpora
import pickle
import gensim


In [ ]:
dictionary = corpora.Dictionary(text_data2)
corpus = [dictionary.doc2bow(text) for text in text_data2]

pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:

NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=30)
ldamodel.save('model3.gensim')


In [ ]:
topics = ldamodel.print_topics(num_words=8)
for topic in topics:
    print(topic)

### Visualizing

In [ ]:
import pyLDAvis.gensim

In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))

In [ ]:
lda = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [ ]:
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=8)
ldamodel.save('model10.gensim')

lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display10 = pyLDAvis.gensim.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)

# spaCy

In [ ]:
text = "El SIELAE consideró que debía aprovechar sus conocimientos de expertos en emblemática para ofrecer a otros investigadores y estudiantes la posibilidad de encontrar rápidamente en la web las ediciones de libros de emblemas, sin tener que pasar horas buscando ediciones diversas, etc. Así, desde 2006, se ofrece un Catálogo de ediciones digitales de libros de emblemas y obras afines accesibles en Internet/DEBOW (Digital Emblem Books on Web) que va por la quinta edición (julio 2013) y que contiene 1.266 fichas de ediciones de libros de emblemas u obras afines en formato digital, con uno o más ejemplares disponibles en Internet, y los enlaces correspondientes para un acceso directo. El número de ejemplares de los que proporcionamos enlace es de 1.934. El fichero en PDF ocupa 191 páginas."

In [ ]:
doc = nlp(text)

In [ ]:
doc[0].pos_